 Briefly describe the recommender system that you’re going to build out from a business perspective

I built a simple recommender system for Data Science Books. This system will take readers input ratings for their familiar data science books and predict the reader's preference for possible unreaded books. This is similar in approach to systems used by Netflix, Hulu and Amazon Prime.

build out my own toy dataset

I have collected data on 5 popular data science books: Machine Learning (Tom Mitchell), The Master of Algorithm (Pedro Domingos), R Cookbook (Paul Teetor), Artificial Intelligence: A Modern Approach(Stuart Russell and Peter Norvig),and Natrual laguage Processing with Python (Steven Bird, Ewan Klein and Edward Loper) . Friends were asked to rate these shows from 1 for 'Awful' to 5 for 'Outstanding' and to leave blank un-read. The data will be used to generate a basic recommendation system for a test set of withheld data and unread books.

Load Data and create a user-item matrix

In [15]:
#import packages to set working environment:
import pandas as pd
from matplotlib import pyplot as plt

In [16]:
bk_df = pd.read_csv('https://raw.githubusercontent.com/johnpannyc/jpan_data612_project1/master/data_science_book_rating.csv')
bk_df = bk_df[sorted(bk_df.columns)]
bk_df.head(6)

,Artificial Intelligence: A Modern Approach,Machine Learning,Natrual laguage Processing with Python,R Cookbook,The Master of Algorithm,Unnamed: 0
0,3.0,2.0,5.0,4,3.0,0
1,NaN,3.0,4.0,5,3.0,1
2,NaN,NaN,5.0,4,3.0,2
3,4.0,3.0,4.0,4,NaN,3
4,3.0,4.0,4.0,5,2.0,4
5,4.0,2.0,NaN,4,4.0,5


In [17]:
bk_df.shape

(6, 6)

In [18]:
#calculate the overall mean of all the dataframe
bk_df_mean = bk_df.stack().mean(skipna = True) #stack makes the DF one column, mean takes the mean of that colunm.
print(bk_df_mean) 
#below are the means for the rows and the columns of the dataframe
bk_df_col_means = bk_df.mean(axis=0,skipna = True)
bk_df_row_means = bk_df.mean(axis=1,skipna = True)
print(bk_df_col_means)
print(bk_df_row_means[:6])

3.4193548387096775
Artificial Intelligence: A Modern Approach    3.500000
Machine Learning                              2.800000
Natrual laguage Processing with Python        4.400000
R Cookbook                                    4.333333
The Master of Algorithm                       3.000000
Unnamed: 0                                    2.500000
dtype: float64
0    2.833333
1    3.200000
2    3.500000
3    3.600000
4    3.666667
5    3.800000
dtype: float64


break the dataset into "train" and "test"

In [19]:
#break the dataset into "train" and "test"
from sklearn.model_selection import train_test_split
bk_col_df = bk_df.stack()
bk_col_df.head(30)

bk_train, bk_test = train_test_split(bk_col_df, test_size =0.2)
bk_train = bk_train.sort_index()
bk_test = bk_test.sort_index()

# calculate the raw average (mean) rating for every user-item combination. 
train_raw_ave = bk_train.mean()
test_raw_ave = bk_test.mean()
print("The Raw Average on the Training Set is: " + str(train_raw_ave) + "\n")
print("The Raw Average on the Test Set is: " + str(test_raw_ave) + "\n")

The Raw Average on the Training Set is: 3.4166666666666665

The Raw Average on the Test Set is: 3.4285714285714284



Recover the data frames into their original wide format and calculate the row and column averages to calculate the biases:

In [20]:
#for train data
bk_train_wide = bk_train.unstack().sort_index()
bk_train_col_means = bk_train_wide.mean(axis=0,skipna = True)
print(str(bk_train_col_means))
bk_train_row_means = bk_train_wide.mean(axis=1,skipna = True)
print(str(bk_train_row_means[:6]))

Artificial Intelligence: A Modern Approach    3.50
Machine Learning                              2.75
Natrual laguage Processing with Python        4.40
R Cookbook                                    4.00
The Master of Algorithm                       3.00
Unnamed: 0                                    2.80
dtype: float64
0    2.800000
1    4.000000
2    3.333333
3    3.600000
4    3.400000
5    3.800000
dtype: float64


In [21]:
#for test data
bk_test_wide = bk_test.unstack().sort_index()
bk_test_wide
bk_test_col_means = bk_test_wide.mean(axis=0,skipna = True)
print(str(bk_test_col_means))
bk_test_row_means = bk_test_wide.mean(axis=1,skipna = True)
print(str(bk_test_row_means[:6]))

The Master of Algorithm    3.000000
Machine Learning           3.000000
R Cookbook                 4.666667
Unnamed: 0                 1.000000
dtype: float64
0    3.0
1    3.0
2    4.0
4    5.0
dtype: float64


Calculate the RMSE for raw average for both my training data and your test data. 

In [22]:
from sklearn.metrics import mean_squared_error

train_pred = [bk_train.mean()]*len(bk_train)
train_pred
train_rmse = mean_squared_error(bk_train, train_pred)**0.5
print("RMSE on the Training Set is: " + str(train_rmse) + "\n")
test_pred = [bk_train.mean()]*len(bk_test) 
test_pred
test_rmse = mean_squared_error(bk_test, test_pred)**0.5
print("RMSE on the Test Set is: " + str(test_rmse) + "\n")

RMSE on the Training Set is: 1.1516895800904379

RMSE on the Test Set is: 1.2936812246883354



Using my training data, calculate the bias for each user and each item. 

In [23]:
#Using my training data, calculate the bias for each user and each item.
show_bias = bk_train_col_means - train_raw_ave 
print(show_bias)
user_bias = bk_train_row_means - train_raw_ave 
print(user_bias[:6]) 

#From the raw average, and the appropriate user and item biases, 
#calculate the baseline predictors for every user-item combination. 
bk_pred = pd.DataFrame(index=bk_df.index, columns=bk_df.columns.values)
for i in range(0,len(user_bias)):
    for j in range(0,len(show_bias)):
            bk_pred.iloc[i,j] = train_raw_ave + user_bias[i] + show_bias[j]
            if bk_pred.iloc[i,j] < 1.0:
                bk_pred.iloc[i,j] = 1.0
            elif bk_pred.iloc[i,j] > 5.0:
                tv_pred.iloc[i,j] = 5.0
bk_pred.head(6)

Artificial Intelligence: A Modern Approach    0.083333
Machine Learning                             -0.666667
Natrual laguage Processing with Python        0.983333
R Cookbook                                    0.583333
The Master of Algorithm                      -0.416667
Unnamed: 0                                   -0.616667
dtype: float64
0   -0.616667
1    0.583333
2   -0.083333
3    0.183333
4   -0.016667
5    0.383333
dtype: float64


,Artificial Intelligence: A Modern Approach,Machine Learning,Natrual laguage Processing with Python,R Cookbook,The Master of Algorithm,Unnamed: 0
0,2.88333,2.13333,3.78333,3.38333,2.38333,2.18333
1,4.08333,3.33333,4.98333,4.58333,3.58333,3.38333
2,3.41667,2.66667,4.31667,3.91667,2.91667,2.71667
3,3.68333,2.93333,4.58333,4.18333,3.18333,2.98333
4,3.48333,2.73333,4.38333,3.98333,2.98333,2.78333
5,3.88333,3.13333,4.78333,4.38333,3.38333,3.18333


In [31]:
import math
bk_pred_train = pd.DataFrame(index=bk_df.index, columns=bk_df.columns.values)
bk_pred_train = bk_pred_train[sorted(bk_pred_train.columns)]
bk_train_wide = bk_train_wide[sorted(bk_train_wide.columns)]
for i in range(0,len(user_bias)):
    for j in range(0,len(show_bias)):
        if math.isnan(bk_train_wide.iloc[i,j]):
            continue
        else:
            bk_pred_train.iloc[i,j] = train_raw_ave + user_bias[i] + show_bias[j]
            if bk_pred_train.iloc[i,j] < 1.0:
                bk_pred_train.iloc[i,j] = 1.0
            elif bk_pred_train.iloc[i,j] > 5.0:
                bk_pred_train.iloc[i,j] = 5.0
bk_pred_train.head(6)

,Artificial Intelligence: A Modern Approach,Machine Learning,Natrual laguage Processing with Python,R Cookbook,The Master of Algorithm,Unnamed: 0
0,2.88333,2.13333,3.78333,3.38333,NaN,2.18333
1,NaN,NaN,4.98333,NaN,NaN,NaN
2,NaN,NaN,4.31667,NaN,2.91667,2.71667
3,3.68333,2.93333,4.58333,4.18333,NaN,2.98333
4,3.48333,2.73333,4.38333,NaN,2.98333,2.78333
5,3.88333,3.13333,NaN,4.38333,3.38333,3.18333


In [44]:
train_base_rmse = mean_squared_error(bk_train, bk_pred_train.stack().sort_index())**0.5
print("RMSE on the Training Set is: " + str(train_bias_rmse) + "\n")

RMSE on the Training Set is: 0.8815233027738593



In [37]:
bk_pred_test = pd.DataFrame(index=bk_df.index, columns=bk_df.columns.values)
bk_pred_test = bk_pred_test[sorted(bk_pred_test.columns)]
bk_test_wide = bk_test_wide[sorted(bk_test_wide.columns)]
bk_test_wide = bk_test_wide.reindex(bk_df.index)

for l in range(0,len(user_bias)):
    for k in range(0,len(show_bias)):
        if math.isnan(bk_test_wide.iloc[l,k]):
            #continue
        else:
            bk_pred_test.iloc[l,k] = test_raw_ave + user_bias[l] + show_bias[k]
              if bk_pred_test.iloc[l,k] < 1.0:
                bk_pred_test.iloc[l,k] = 1.0
            elif bk_pred_test.iloc[l,k] > 5.0:
                bk_pred_test.iloc[l,k] = 5.0
bk_pred_test.head(6)

,Machine Learning,R Cookbook,The Master of Algorithm,Unnamed: 0
0,NaN,NaN,3.0,NaN
1,3.0,5.0,3.0,1.0
2,NaN,4.0,NaN,NaN
3,NaN,NaN,NaN,NaN
4,NaN,5.0,NaN,NaN
5,NaN,NaN,NaN,NaN


In [46]:
test_base_rmse = mean_squared_error(bk_test, bk_pred_test.stack().sort_index())**0.5
print("RMSE on the Test Set is: " + str(test_bias_rmse) + "\n")

Summary

Lets calculate the percentage improvements based on the original (simple average) and baseline predictor (including bias) RMSE numbers for both Test and Train data sets.

The results show that we see a 23.47% improvement in making a prediction for the ratings in the Training data set. Where as we see only 26.358% improvement in prediction for the Test data set. Both are positive hoewver the Training data set yielded better prediction.


In [4]:

# Train data set

Per_Improv_Train = (1-(0.88/1.15))*100  #(1-(RMSE_BASE_TRAIN/RMSE_TRAIN))*100
Per_Improv_Train



23.478260869565215

In [5]:
# Test data set


Per_Improv_Test = (1-(0.95/1.29))*100  ## (1-(RMSE_BASE_TEST/RMSE_TEST))
Per_Improv_Test



26.356589147286826